In [1]:
using PyCall
using Plots
using Distributed

In [2]:
@pyimport tensorflow as tf

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data();

In [4]:
strain=size(x_train);
stest= size(x_test);

In [5]:
x_train_reshape = reshape(x_train[1:5000,:,:],(5000,strain[2],strain[3],1))
x_test_reshape = reshape(x_test[1:1000,:,:],(1000,stest[2],stest[3],1))
size(x_train_reshape),size(x_test_reshape)

((5000, 28, 28, 1), (1000, 28, 28, 1))

In [6]:
y_train_cat=tf.keras.utils.to_categorical(y_train[1:5000],length(unique(y_train[1:5000])))
y_test_cat =tf.keras.utils.to_categorical(y_test[1:1000],length(unique(y_test[1:1000])));

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(8,(3,3),activation="relu",strides=2,input_shape=(size(x_train_reshape)[2],
        size(x_train_reshape)[3],1)))
model.add(tf.keras.layers.Conv2D(16,(3,3),activation="relu"))
model.add(tf.keras.layers.Conv2D(32,(3,3),activation="relu"))
model.add(tf.keras.layers.GlobalMaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation("softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

2021-01-06 14:08:20.652568: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 8)         80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          4640      
_________________________________________________________________
global_max_pooling2d (Global (None, 32)                0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                330       
_________________________________________________________________
activation (Activation)      (None, 10)                0

In [9]:
@time history=model.fit(x_train_reshape,y_train_cat,epochs=10,batch_size=128,validation_data=(x_test_reshape,y_test_cat))

Epoch 1/10
40/40 [==============================] - 0s 11ms/step - loss: 17.2614 - accuracy: 0.1204 - val_loss: 3.2854 - val_accuracy: 0.1480
Epoch 2/10
40/40 [==============================] - 0s 9ms/step - loss: 2.5588 - accuracy: 0.2340 - val_loss: 2.1985 - val_accuracy: 0.3070
Epoch 3/10
40/40 [==============================] - 0s 8ms/step - loss: 1.9104 - accuracy: 0.3754 - val_loss: 1.7698 - val_accuracy: 0.4190
Epoch 4/10
40/40 [==============================] - 0s 8ms/step - loss: 1.5073 - accuracy: 0.5052 - val_loss: 1.5286 - val_accuracy: 0.4950
Epoch 5/10
40/40 [==============================] - 0s 8ms/step - loss: 1.2301 - accuracy: 0.5978 - val_loss: 1.3490 - val_accuracy: 0.5570
Epoch 6/10
40/40 [==============================] - 0s 7ms/step - loss: 1.0409 - accuracy: 0.6634 - val_loss: 1.2071 - val_accuracy: 0.6060
Epoch 7/10
40/40 [==============================] - 0s 7ms/step - loss: 0.8980 - accuracy: 0.7130 - val_loss: 1.0918 - val_accuracy: 0.6550
Epoch 8/10
40/40 [

  4.109295 seconds (1.15 M allocations: 58.861 MiB)


PyObject <tensorflow.python.keras.callbacks.History object at 0x00000000587A7D60>

In [35]:
JULIA_NUM_THREADS=4;

In [36]:
@time history1=model.fit(x_train_reshape,y_train_cat,epochs=10,batch_size=128,validation_data=(x_test_reshape,y_test_cat));

Epoch 1/10
40/40 [==============================] - 0s 9ms/step - loss: 6.8843e-04 - accuracy: 1.0000 - val_loss: 0.6078 - val_accuracy: 0.8870
Epoch 2/10
40/40 [==============================] - 0s 8ms/step - loss: 7.3353e-04 - accuracy: 1.0000 - val_loss: 0.6079 - val_accuracy: 0.8890
Epoch 3/10
40/40 [==============================] - 0s 7ms/step - loss: 8.3666e-04 - accuracy: 1.0000 - val_loss: 0.6087 - val_accuracy: 0.8880
Epoch 4/10
40/40 [==============================] - 0s 8ms/step - loss: 6.5527e-04 - accuracy: 1.0000 - val_loss: 0.6138 - val_accuracy: 0.8860
Epoch 5/10
40/40 [==============================] - 0s 8ms/step - loss: 6.3545e-04 - accuracy: 1.0000 - val_loss: 0.6114 - val_accuracy: 0.8920
Epoch 6/10
40/40 [==============================] - 0s 8ms/step - loss: 5.8811e-04 - accuracy: 1.0000 - val_loss: 0.6135 - val_accuracy: 0.8900
Epoch 7/10
40/40 [==============================] - 0s 8ms/step - loss: 5.8440e-04 - accuracy: 1.0000 - val_loss: 0.6146 - val_accuracy:

  3.247499 seconds (68 allocations: 3.984 KiB)


In [ ]:
loss = hcat(history.history["loss"],history.history["val_loss"])
acc = hcat(history.history["accuracy"],history.history["val_accuracy"])
plot(plot(1:10,loss,labels=["Training Loss" "Validation Loss"]),plot(1:10,acc,labels=["Training Accuracy" "Validation Accuracy"]))

In [37]:
acc0 = model.evaluate(x_test_reshape,y_test_cat)

32/32 [==============================] - 0s 2ms/step - loss: 0.6191 - accuracy: 0.8930


2-element Array{Float64,1}:
 0.6191431283950806
 0.8930000066757202

In [ ]:
acc1 = 